# 1. Install Required Libraries
(Skip if already installed)

In [ ]:
# Uncomment below if needed:
# !pip install pandas openpyxl

# 2. Define Data Classes

In [ ]:
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team


# 3. Load Cleaned Excel Files

In [ ]:

import pandas as pd

# Update paths to where you saved your cleaned files
path_2023 = '/home/justin/JustInternetAI/Calcutta/Data/2023_cleaned_events_fixed_V2.xlsx'
path_2024 = '/home/justin/JustInternetAI/Calcutta/Data/2024_cleaned_events_V2.xlsx'

sheets_2023 = pd.ExcelFile(path_2023).sheet_names
sheets_2024 = pd.ExcelFile(path_2024).sheet_names


# 4. Build Player and Tournament Structures

In [ ]:

players = {}
tournaments = {}

def get_or_create_player(player_name):
    if player_name not in players:
        players[player_name] = Player(player_name)
    return players[player_name]

def load_event(file_path, sheet_name, year):
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    tournament_name = f"{year} {sheet_name}"  # Full name including year
    tournament = Tournament(tournament_name)
    round_obj = Round(tournament_name, round_number=1)

    for _, row in df.iterrows():
        player = get_or_create_player(row['Player'])
        hole_scores = [row[i] for i in range(4, 22)]

        player_round = PlayerRoundInfo(
            player=player,
            tournament_name=tournament_name,
            round_number=1,
            handicap=row.get('Handicap'),
            tee=row['Tee'],
            hole_scores=hole_scores,
            total=row['Total'],
            net=row['Net']
        )

        player.rounds.append(player_round)
        round_obj.player_rounds.append(player_round)

    tournament.rounds.append(round_obj)
    tournaments[tournament_name] = tournament


# 5. Load All Events

In [ ]:

# Load all 2023 events
for sheet in sheets_2023:
    load_event(path_2023, sheet, year=2023)

# Load all 2024 events
for sheet in sheets_2024:
    load_event(path_2024, sheet, year=2024)


In [ ]:

# List all players
print(f"Loaded {len(players)} players.")
print(list(players.keys())[:10])  # Show first 10 players

# List all tournaments
print(f"Loaded {len(tournaments)} tournaments.")
print(list(tournaments.keys())[:5])

# Loop through all players and print their name and all rounds they played
for player_name, player_obj in players.items():
    print(f"\n{player_name} played {len(player_obj.rounds)} rounds:")
    for round_info in player_obj.rounds:
        print(f"- {round_info.tournament_name} (Net {round_info.net})")

# 6. Save Player and Tournaments to a file

In [ ]:


import pickle

# Save players and tournaments to a file
def save_data(players, tournaments, filename='golf_data.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump({'players': players, 'tournaments': tournaments}, f)

# Load players and tournaments from a file
def load_data(filename='golf_data.pkl'):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data['players'], data['tournaments']

In [ ]:
save_data(players, tournaments)
loaded_players, loaded_tournaments = load_data()
print(f"Loaded {len(loaded_players)} players from file.")

In [ ]:
# --- Helper to find corrupted rounds ---

def find_corrupt_rounds(players):
    bad_rounds = []
    player_list = players.values() if isinstance(players, dict) else players

    for player in player_list:
        for rnd in player.rounds:
            for score in rnd.hole_scores:
                if not isinstance(score, (int, float)):
                    try:
                        float(score)
                    except (ValueError, TypeError):
                        bad_rounds.append((player.name, score))
    return bad_rounds

# Example usage
corrupt = find_corrupt_rounds(players)
for player_name, bad_score in corrupt:
    print(f"Corrupt score '{bad_score}' found for player {player_name}")

In [ ]:


# --- Helper to Print Sections of Player Data ---

def inspect_player_rounds(players, start_idx=0, num_players=5):
    """
    Print sections of player data to manually inspect hole_scores.
    Args:
        players: dict or list of Player objects
        start_idx: starting index in the player list
        num_players: how many players to show
    """
    player_list = list(players.values()) if isinstance(players, dict) else list(players)
    
    for i, player in enumerate(player_list[start_idx:start_idx+num_players], start=start_idx):
        print(f"\n=== Player {i}: {player.name} ===")
        for j, round_info in enumerate(player.rounds):
            print(f"  Round {j}: Tournament={round_info.tournament_name}, RoundNum={round_info.round_number}, Handicap={round_info.handicap}")
            print(f"    Hole Scores: {round_info.hole_scores}")
            
            # Check if any gross score is suspicious
            for k, score in enumerate(round_info.hole_scores):
                try:
                    _ = float(score)
                except (ValueError, TypeError):
                    print(f"    ⚠️ Suspicious score at hole {k+1}: '{score}'")



In [ ]:
# See first 5 players
inspect_player_rounds(players, start_idx=0, num_players=5)

# See players 10–15
#inspect_player_rounds(players, start_idx=10, num_players=5)

# See players 50–60
#inspect_player_rounds(players, start_idx=50, num_players=10)

In [ ]:
# --- Inspect a Player's Rounds in Full Detail ---

def inspect_player_stableford_rounds(players, player_name):
    """
    Print out detailed hole-by-hole gross, net, and Stableford points for each round of a player.
    """
    player = players.get(player_name) if isinstance(players, dict) else None
    if not player:
        print(f"Player '{player_name}' not found.")
        return
    
    print(f"\n=== Rounds for {player.name} ===\n")
    
    for rnd_idx, rnd in enumerate(player.rounds):
        print(f"--- Round {rnd_idx + 1}: Tournament={rnd.tournament_name}, Round={rnd.round_number}, Handicap={rnd.handicap} ---")
        
        strokes = strokes_received_per_hole(rnd.handicap)
        gross_scores = rnd.hole_scores
        
        net_scores = []
        stableford_scores = []
        
        gross_total = 0
        net_total = 0
        stableford_total = 0

        print(f"{'Hole':<5} {'Gross':>8} {'Net':>8} {'Stableford':>12}")
        print("-" * 35)
        
        for i in range(18):
            try:
                gross = float(gross_scores[i])
            except (ValueError, TypeError, IndexError):
                gross = None
            
            if gross is None:
                net = None
                stableford = None
            else:
                net = gross - strokes[i]
                net_relative_to_par = round(net - hole_pars[i])  
                stableford = stableford_points(net_relative_to_par)
            
            gross_str = f"{gross:.0f}" if gross is not None else "X"
            net_str = f"{net:.0f}" if net is not None else "X"
            stableford_str = f"{stableford:+d}" if stableford is not None else "X"

            print(f"{i+1:<5} {gross_str:>8} {net_str:>8} {stableford_str:>12}")

            if gross is not None:
                gross_total += gross
            if net is not None:
                net_total += net
            if stableford is not None:
                stableford_total += stableford
        
        print("-" * 35)
        print(f"{'Total':<5} {gross_total:>8.0f} {net_total:>8.0f} {stableford_total:>12.0f}\n")